In [ ]:
import import_ipynb
from bottles_canny import binary_mask
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir, makedirs
from os.path import isfile, join, dirname, realpath, isdir
import glob2


def extract_ROI(picture, cX,cY, roi):

    if cY-64 <0:
        cY= 64
    elif cY+64 > picture.shape[0]:
        cY = picture.shape[0] - 64
    if cX-64 < 0:
        cX = 64
    elif cX+64 > picture.shape[1]:
        cX = picture.shape[1] - 64
    roi = np.dstack((roi , (picture[cY-64:cY+64 , cX-64: cX+64, :])))
    
    return roi

def ROI_extr(mask,image, p ):
    k=0
    rois = np.zeros((128,128,0))
    coords = np.empty((0,2))
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        cor=-1
        if k > 0:
            for i in range(coords.shape[0]):
                if cX >= (coords[i,1] - 128) and cX <= (coords[i,1] + 128):
                    if cY >= (coords[i,0] - 128) and cY <= (coords[i,0] + 128):
                        cor = 1
                        break
        if cor == -1 or k==0:
            coords = np.vstack((coords, [cY,cX]))
            roi = np.zeros((128,128,0))
            roi = extract_ROI(image, cX, cY,roi)
            for dd in range(1,(p+1)):
                hh = dd*32
                cxx = cX-(hh)
                roi = extract_ROI(image,cxx,cY,roi)
                cyy = cY - (hh)
                roi = extract_ROI(image, cX, cyy,roi)
                roi = extract_ROI(image,cxx,cyy,roi)
                cxx = cX + (hh)
                roi = extract_ROI(image,cxx,cY,roi)
                roi = extract_ROI(image,cxx,cyy,roi)
                cyy = cY + (hh)
                roi = extract_ROI(image,cX, cyy,roi)
                roi = extract_ROI(image, cxx,cyy,roi)
                cxx = cX-hh
                roi = extract_ROI(image, cxx,cyy,roi)
            rois = np.dstack((rois,roi))
            #cv2.circle(image, (cX, cY), 20, (255, 255, 255), 5)
            k=k+1
    #plt.figure(figsize=(20,10))
    #plt.imshow(image)
    #print(rois.shape)
    return rois

